In [643]:
###############################################################
# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)
###############################################################

# 1. İş Problemi (Business Problem)
# 2. Veriyi Anlama (Data Understanding)
# 3. Veri Hazırlama (Data Preparation)
# 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
# 5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
# 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
# 7. Tüm Sürecin Fonksiyonlaştırılması

###############################################################
# 1. İş Problemi (Business Problem)
###############################################################

# Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre
# pazarlama stratejileri belirlemek istiyor.

# Veri Seti Hikayesi
#🔗  https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
# 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.
# Hediyelik eşya satıyor-toptancı - müşteriler kurumsal

# Veri setindeki Değişkenler
#
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.


###############################################################
# 2. Veriyi Anlama (Data Understanding)
###############################################################

import datetime as dt  
import pandas as pd
pd.set_option('display.max_columns', None)  #!!yazdırma işleminde tüm sütunlar görünmesi için
# pd.set_option('display.max_rows', None)   #!!yazdırma işleminde tüm satırların görünmesi için
pd.set_option('display.float_format', lambda x: '%.3f' % x)  #!sayısal değişkenlerin , sonrası kaç basamagı görünsün

df_ = pd.read_excel("C:/Users/Sedef/Customer-Segmentation-with-RFM/datasets/online_retail_II.xlsx", sheet_name="Year 2009-2010")

df = df_.copy()  #kopyası üzerinde çalışırız ilerde -veri okuma uzun sürdügünden pratikleştirildi


In [644]:
df.head() #1_result


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [645]:
df.shape  #2_result


(525461, 8)

In [646]:
df.isnull().sum()  #3_result #eksik değer var mı - hangi alanda ne kadar?


Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [647]:
df["Description"].nunique() #4_result # essiz urun sayisi nedir?

4681

In [648]:
df["Description"].value_counts().head() #5_result #Hangi üründen kaçar tane var/faturalandırıldı/satıldı


Description
WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: count, dtype: int64

In [649]:
df.groupby("Description").agg({"Quantity": "sum"}).head() #!! - değer geldi veri ön işlemde bakılacak

,Quantity
Description,
21494,-720
22467,-2
22719,2
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690


In [650]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head() #6_result #Hangi üründen toplam kaçar sayış yapıldı sıralaması

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [651]:
df["Invoice"].nunique()  #7_result #Toplam kaç eşsiz fatura kesildi

28816

In [652]:
#8_result #Fatura başına tolpam ne kadar kazanıldı
df["TotalPrice"] = df["Quantity"] * df["Price"]  
df.head()
df.groupby("Invoice").agg({"TotalPrice": "sum"}).head() 

,TotalPrice
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240


In [653]:
###############################################################
# 3. Veri Hazırlama (Data Preparation)
###############################################################

df.shape  #9_result


(525461, 9)

In [654]:
df.isnull().sum()  #10_result


Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

In [655]:
#11_result 
df = df[(df['Quantity'] > 0)]  #Quantity min değeri 1 olacak şekilde filtrelendi
df.dropna(inplace=True)        
df.shape


(407695, 9)

In [656]:
df.describe().T    #12_result   #transpozu alındı okunabilirlik artsın diye

,count,mean,min,25%,50%,75%,max,std
Quantity,407695.000,13.587,1.000,2.000,5.000,12.000,19152.000,96.842
InvoiceDate,407695,2010-07-01 10:10:10.782177792,2009-12-01 07:45:00,2010-03-26 14:01:00,2010-07-09 15:46:00,2010-10-14 17:09:00,2010-12-09 20:01:00,NaN
Price,407695.000,3.294,0.000,1.250,1.950,3.750,10953.500,34.757
Customer ID,407695.000,15368.504,12346.000,13997.000,15321.000,16812.000,18287.000,1679.796
TotalPrice,407695.000,21.663,0.000,4.950,11.900,19.500,15818.400,77.147


In [657]:

#📌!! ~ dışındakileri getir demek için kullanılır
#invoice için başında C olanlar iptallerdi -iadeler veri seti yapısını bozuyor-dışarda bırakılsın
#df = df[~df["Invoice"].str.contains("C", na=False)]  #asil kod bu ama calismadi
df = df[~df["Invoice"].astype(str).str.startswith("C")]

In [658]:
###############################################################
# 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
###############################################################

# 📌Her müşteri özelinde Recency, Frequency, Monetary hesaplanması
#Recency --> müşterinin yeniliği --> (analiz tarihi-müşteri son satın alma tarihi)
#Frequency --> müşterinin toplam satın alması
#Monetary  --> müşterinin satın almasının parsal karşılığı
df.head() #13_result
#hesaplamalar için veri setine bir analiz tarihi(gün) tanımlanmalı
#📢 !! not daha önce import edildi - (import datetime as dt )


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [659]:
#14_result
df["InvoiceDate"].max() 


Timestamp('2010-12-09 20:01:00')

In [660]:
today_date = dt.datetime(2010, 12, 11)
type(today_date)


datetime.datetime

In [661]:

#15_result
#📌 aggrigation fonksiyonu içine verilen sözlük yapısında key kısmında değişkenler ve value kısmına da 
#bu değişkenlere uygulamak istediğin fonksiyonları ifade edersen iligli fonksiyonlara uygular
#💡Kullan at fonsiyon tanımlanacağından lambda kullanıldı
	#her müşterinin recency eğeri için
	#frequency değeri için - eşsiz fatura
	#Monetary değeri için toplam totalprice
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days, 
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.head() 


,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [662]:
#Değişken isimleri yeniden tanımlandı
rfm.columns = ['recency', 'frequency', 'monetary'] 
rfm.head() 
rfm.describe().T  #mantıklı mı sonuçlar incele bakalım +Monetary 0.00 kaldıırlsın


,count,mean,std,min,25%,50%,75%,max
recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [663]:

#16_result
#Monetary 0.00 kaldıırlsın
rfm = rfm[rfm["monetary"] > 0]
rfm.describe().T 


,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [664]:

#17_result
rfm.shape  #yeni veri setindeki müşteri sayııs- metrikler

(4312, 3)

In [665]:
###############################################################
# 5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
###############################################################
#📢 hatırlatma : Recency de küçük olan değerli M ve F de büyük olanlar iyi

#📌 qcut fonksiyonu değişkeni küçükten büyüğe sıralar ve çeyrek değerlere göre bölme işlemi yapar
#verilen değişkeni kaç parçaya bölünmesi istendiği ve sonrasında label/etiket bilgisini alır
#💡 bu aralıklarda değişen değişkenkler olsun.ör:yaş çocuk - genç... olarak bölünerek gruplanabilir
# 0-100, 0-20, 20-40, 40-60, 60-80, 80-100

#18_result
#⚠️recency_score için label sıralaması diğerlerinden ters-küçük olan iyidir olarak skorlandı
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
#⚠️  error-oluşturulan aralıklarda unique değerler yer almamaktadır- value error
# çok fazla tkerar eden frenkans var ve küçükten büyüğe sırlaandığında çeyrek değrlere düşen değerle aynı olmuş
#çözüm 📌 rank methodu ile ilk gördüğünü ilk sınıfa ata diyerek bu problem çözüldü
#rfm["frequency_score"] = pd.qcut(rfm['frequency'], 5, labels=[1, 2, 3, 4, 5])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

In [666]:
#19_result
#📢 hatırlatma RFM score için R ve F yeterli denmişti
#önce string çevirip -toplayıp-yeni değer atandı
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

In [667]:

#20_result
#⚠️string olduğundan RFM_sCORE  sayısal değişken gibi analiz edilmedi
rfm.describe().T


,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [668]:

#21_result
#şampiyon sınıfı 55 - şampiyonlar kim  en değerli müşteriler
rfm[rfm["RFM_SCORE"] == "55"]

rfm[rfm["RFM_SCORE"] == "11"]

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12355.000,203,1,488.210,1,1,2,11
12362.000,374,1,130.000,1,1,1,11
12366.000,269,1,500.240,1,1,2,11
12368.000,264,1,917.700,1,1,3,11
12378.000,198,1,1407.700,1,1,4,11
...,...,...,...,...,...,...,...
15928.000,292,1,293.530,1,1,2,11
15929.000,280,1,594.000,1,1,3,11
15941.000,273,1,405.000,1,1,2,11


In [669]:
###############################################################
# 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
###############################################################
# 📌regex - reguler expression -ek ders var ⭐

# RFM isimlendirmesi - sınıflar ve skor için
#💡ilk satır için birinci elemanında 1 veya 2 ve 2. elemanında 1 veya iki görürsen hibernating isimlendirmesi yap
#55  champions örnek
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

#veri setine bu tanımları eklemek için replace(key_value,) kullanıldı
#yeni değişken segment eklendi
#regex=True -->
#22_result
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()#benekledimciktigörünsündiye

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,5,2,25,cant_loose
12347.000,3,2,1323.320,5,2,4,52,potential_loyalists
12348.000,74,1,222.160,2,1,1,21,hibernating
12349.000,43,3,2671.140,3,3,5,33,need_attention
12351.000,11,1,300.930,5,1,2,51,new_customers


In [670]:
#23_result
#Segment analizi için temel bilgiler
# ⚠️Bunlar metrikler skorlar değil -ortalamalarını alarak karşılaştırma yapılacak segmentler arasında
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])


recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_Risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [671]:

#24_result
#📢 analiz sonrası odaklanılması gereken sınıflara özel çalışma yapılmalı
rfm[rfm["segment"] == "cant_loose"].head()


,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,5,2,25,cant_loose
12380.000,101,7,6951.490,2,5,5,25,cant_loose
12482.000,212,29,23691.400,1,5,5,15,cant_loose
12510.000,95,7,4195.450,2,5,5,25,cant_loose
12891.000,94,8,509.500,2,5,3,25,cant_loose


In [672]:
rfm[rfm["segment"] == "cant_loose"].index  #müşterilerin index -id bilgisi geli


Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0, 13313.0,
       13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0, 14160.0, 14221.0,
       14548.0, 14607.0, 14685.0, 14745.0, 15003.0, 15013.0, 15015.0, 15125.0,
       15141.0, 15222.0, 15306.0, 15321.0, 15359.0, 15369.0, 15372.0, 15443.0,
       15538.0, 15607.0, 15633.0, 15722.0, 15751.0, 15754.0, 15768.0, 15911.0,
       15912.0, 16027.0, 16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0,
       16631.0, 16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
       17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0, 17454.0,
       17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0, 17988.0, 18009.0,
       18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
      dtype='float64', name='Customer ID')

In [673]:

#25_result
#ilgili departman anlayacağı dile çevir
new_df = pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index
new_df.head()#benekledimciktigörünsündiye

,new_customer_id
0,12351.000
1,12385.000
2,12386.000
3,12427.000
4,12441.000


In [674]:

#📌 astype(int) .000 lı id değerlerinden kurtardı
new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)
new_df.head()#benekledimciktigörünsündiye


,new_customer_id
0,12351
1,12385
2,12386
3,12427
4,12441


In [675]:

#ilgili departmana iletmek için csv ye çevir - veya bir db ye aktarılabilir be BI araçları ile görselleşiirlebilir
new_df.to_csv("new_customers.csv") #26_result


In [676]:

#rfm de csv olarak çıkarılabilir- bu yöntemde tüm segment bilgileri gider
rfm.to_csv("rfm.csv")